In [ ]:
#| label: libraries
#| include: false
import pandas as pd
import numpy as np
import plotly.express as px
import sqlite3

## Elevator pitch

__This project reports the analysis of basball database to know various changes in basball and its players. I have used the data collected from 1800s to the present day. I have tried finding of players who attended BYU-Idaho and their salaries. Similarly, I have looked at the average number of hits per players. To conclude, I looked into two teams- New York Yankees(NYA) and Boston Red Sox(BOS) and comapred the wins of each teams throughout the year.


In [ ]:
# Include and execute your code here
sql_file = 'lahmansbaseballdb.sqlite'
connect = sqlite3.connect(sql_file)

__Highlight the Questions and Tasks__

## QUESTION|TASK 1

__Write an SQL query to create a new dataframe about baseball players who attended BYU-Idaho. The new table should contain five columns: playerID, schoolID, salary, and the yearID/teamID associated with each salary. Order the table by salary (highest to lowest) and print out the table in your report.__

__This code creates a table of baseball players who attended BYU-Idaho along with their salary, year and team. The highest paid player goes by 'lindsman01'. The player had a salary of $4 million in 2014 who plays for CHA team. Similarly, player 'stephga01' earned around $1.03 million in 2001. The data shows change on their salaries are the results of change in team and year as well. 


In [ ]:
#| label: Q1
#| code-summary: Read and format data
# Include and execute your code here
my_sql = """ SELECT DISTINCT bat.playerID,
            col.schoolID,
            sal.salary,
            sal.yearID,
            sal.teamID
            FROM batting AS bat
            JOIN salaries AS sal
            ON sal.playerID = bat.playerID
            JOIN collegeplaying AS col 
            ON bat.playerID = col.playerID
            WHERE col.schoolID = 'idbyuid'
            ORDER BY sal.salary DESC; """  

mydata = pd.read_sql_query(my_sql, connect)
mydata

## QUESTION|TASK 2

__This three-part question requires you to calculate batting average (number of hits divided by the number of at-bats)__

__Write an SQL query that provides playerID, yearID, and batting average for players with at least 1 at bat that year. Sort the table from highest batting average to lowest, and then by playerid alphabetically. Show the top 5 results in your report.__

_ This is the first part of the question. It shows the players with batting averahe of 1.0. The findings tells that the player hits the ball every time they were to bat. We limited the data to 5 players only._


In [ ]:
#| label: Q2
#| code-summary: Read and format data
# Include and execute your code here
my_sql = """ SELECT bat.playerID, 
                    bat.yearID,
                    CAST(SUM(bat.H) as FLOAT )/ CAST(SUM(bat.AB) as FLOAT) AS AVG_batting
                    FROM batting AS bat
                    GROUP BY bat.playerID, bat.yearID
                    HAVING SUM(bat.AB) >= 1
                    ORDER BY AVG_batting DESC, bat.playerID
                    LIMIT 5; """


mydata = pd.read_sql_query(my_sql, connect)
mydata

__Use the same query as above, but only include players with at least 10 at bats that year. Print the top 5 results.__

_In the second part of the question, we looked into palyers who had at least 10 at-bats. We found that in 1974, player 'nymanny01' has an average of 0.642857, which is more than half. He seems to hit 6 out of 10 balls. In the top 5 players, everyone was able to hit more than 5  out of 10._


In [ ]:
my_sql = """ SELECT bat.playerID, 
                    bat.yearID,
                    CAST(SUM(bat.H) as FLOAT )/ CAST(SUM(bat.AB) as FLOAT) AS AVG_batting
                    FROM batting AS bat
                    GROUP BY bat.playerID, bat.yearID
                    HAVING SUM(bat.AB) >= 10
                    ORDER BY AVG_batting DESC, bat.playerID
                    LIMIT 5; """


mydata = pd.read_sql_query(my_sql, connect)
mydata

__Now calculate the batting average for players over their entire careers (all years combined). Only include players with at least 100 at bats, and print the top 5 results.__

_This is the final part of the qyestion. In this part, I calculated the batting average for players over their entire careers who had at least 100 at bats. Overall, all player 'cobbty01' seems to be the top batsman in baseball with an average of '0.366299' batting. However, all top 5 players are very close. All these players were skilled in batting._ 


In [ ]:
my_sql = """ SELECT bat.playerID, 
                    CAST(SUM(bat.H) as FLOAT )/ CAST(SUM(bat.AB) as FLOAT) AS AVG_batting
                    FROM batting AS bat
                    GROUP BY bat.playerID
                    HAVING SUM(bat.AB) >= 100
                    ORDER BY AVG_batting DESC, bat.playerID
                    LIMIT 5; """

mydata = pd.read_sql_query(my_sql, connect)
mydata

_Overall, the findings was done to see the batting skills of the players. This result shows us the capability of the players and what year was their best perforamnces._

## QUESTION|TASK 3

__Pick any two baseball teams and compare them using a metric of your choice (average salary, home runs, number of wins, etc). Write an SQL query to get the data you need, then make a graph using Plotly Express to visualize the comparison. What do you learn?__

_This line graph shows the wins of two teams- New York Yankees(NYA) and Boston Red Sox(BOS) in each year. It looks like both the teams are neck-to-neck with each other. However, around 1920 - 1939 and around 1951 - 1963, NYA had more wins than BOS. Both the tems have similar number of wins other years. NYA had an average of 114 wins in 1998. The teams are competitive but, NYA is constant with their average wins where as there are many ups and downs for BOS in their average wins. The data helps us to find their overall performances throughout the years. _


In [ ]:
my_sql = """ SELECT teamID, yearID AS year, AVG(W) AS AVG_wins
              FROM teams
              WHERE teamID IN ('NYA', 'BOS')
              GROUP BY teamID, year; """

mydata = pd.read_sql_query(my_sql, connect)
mydata

chart = px.line(
    mydata,
    x='year',
    y='AVG_wins',
    color='teamID',
    markers=True,
    labels={'AVG_wins': 'Average Wins', 'year': 'Year'},
    title='Average Wins for NYA and BOS Over the Years')

chart.show()